# Project 2:  Lucid Titanic Sleuthing

## Part 1: Developing an understanding of the data

#### Based on the description of the data you read in the readme describe in your own words this data.

Datasets describing the passengers and crew of 2 ship wrecks have been presented.  The datasets contain biographical data, details of the passage on the ship and survival data. 

#### Based on our conceptual understanding of the columns in this data set.  What are the reasonable range of the values for the Sex, Age, SibSp, Parch columns.

The range for sex/gender is binary; it is either male or female.

The Age range is a few months to 85 years old.  This range will be confirmed in the initial data inspection. 

The SibSp column describes family members of the same generation.  Expected values are 0 to 10. There should be peaks at 0 and 1 for single and married travelers.

The ParCh column describes intergeneration connections.  The range should be 0 to 10.  There should be peaks at 0 and 2 for unattached travelers and chilren.  

### Open the data in sublime text is there anything that jumps out to you?

While the datasets describe similar information, the presetnation of the data is very different.  The titanic data set has been encoded and cleaned for analysis.  The Lusitania dataset is contains more data as words and descriptions.  

The contents of the datasets are different, as well.  The Lusitania dataset includes the crew where the Titanic dataset only refers to the passengers and a few employees of White Staar lines or the shipbuilder, Harland and Wolff.

## Part 2: reading the data in

#### Now read the data into a Pandas DataFrame

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as sp
import matplotlib as ply
%matplotlib inline
import seaborn as sns
from __future__ import print_function


In [2]:
titanic_df_raw = pd.read_csv('assets/titanic.csv')
lusitania_df_raw = pd.read_csv('assets/lusitania.csv')

#### Check that the age column doesn't have any unreasonable values 

The titanic age column is automatically converted to floatint points, which indicates there are all the data is numeric.  The range is 0.17 to 80 years. There are many missing values in the age column which are treated as NaN by pandas.  The missing values age values come primarily from the third class and those that embarked in Queenstown, Ireland.  

In [3]:
missing_age = titanic_df_raw.groupby(['embarked']).count()[['name','age']]
missing_age['%Missing'] = (missing_age['name']-missing_age['age'])/missing_age['name']
missing_age = titanic_df_raw.groupby(['pclass']).count()[['name','age']]
missing_age['%Missing'] = (missing_age['name']-missing_age['age'])/missing_age['name']

#repeat for pclass columns

The Lusitania age data has number of values that can not be converted to a numeric value.  Along with missing data, there are many values that indicate uncertainity in the reported value, or report a value in months.  These values will need to be cleaned and converted to an appropriate years value.

#### Check for missing values.  How do you know that a value is missing?

For both datasets, the missing values in the age column are converted to the pandas value 'NaN'.

#### Does it makes sense to guess at the value?

Guessing would not make sense without further information.  The range of ages in datasets is approximatly 0-80 years old.  Further grouping the data, may make guessing at ages a viable choice.

## Part 3: data imputation

#### Well let’s say that it does... You likely noticed that Age has some missing values. How many are missing?

Missing Age Values from Titanic:   263
Missing Age Values from Lusitania: 653

In [4]:
missing_titan_age = sum(list(titanic_df_raw['age'].isnull()))
missing_lusit_age = sum(list(lusitania_df_raw['Age'].isnull()))
missing_titan_age, missing_lusit_age


(263, 653)

#### For the Age of the passengers ... how would you guess at the missing values using the other data present in the CSV.

I would like to break the passengers into subsets. If the mean age of the subset is significantly different from the age of the population, the missing value can be updated to the mean of the subset.  If the subset mean is not signifcantly different or if the subset size is too small, I would reassign the mean of the population.

Cleaning the Titanic data

In [5]:
print ("Titanic dataset column names:") 
print (titanic_df_raw.columns)


Titanic dataset column names:
Index([u'pclass', u'survived', u'name', u'sex', u'age', u'sibsp', u'parch',
       u'ticket', u'fare', u'cabin', u'embarked', u'boat', u'body',
       u'home.dest'],
      dtype='object')


In [6]:
#Mean And Standard Deviation for ages
print ("Mean age of passengers: All Passengers: " +
       format(round(titanic_df_raw['age'].mean(),2)) +
      "  Standard Deviation: All Passengers: " +
       format(round(titanic_df_raw['age'].std(),2)))

chisquare_table = pd.DataFrame(columns=["Variable",'value','mean','std','count','chisquared','p_value'])
chisquare_table.loc[0]=['Population',np.nan,titanic_df_raw['age'].mean(),
                         titanic_df_raw['age'].std(),
                         titanic_df_raw['age'].count(),
                         np.nan,np.nan]
#agg_funcs = {'mean':np.mean, 'std_dev':np.std, 'count':np.count}

# summary_df = titanic_df_raw.pivot_table(columns=['pclass'],aggfunc={'mean', 'count'},values=['age'])
# temp_df = summary_df.transpose().reset_index()
# temp_df['Variable'] = temp_df.columns[0][0]
# temp_df


Mean age of passengers: All Passengers: 29.88  Standard Deviation: All Passengers: 14.41


In [7]:
def make_z_test_table(df, values_col, test_columns_list): 
    import numpy as np
    import pandas as pd
    import scipy.stats as sp
    
    u0 = df[values_col].mean()
    sigma = df[values_col].std()
    median_val = df[values_col].median()
    print ('Median of '+ format(values_col)+ ': '+ format(round(median_val,2)))
    print ('Mean of '+ format(values_col)+ ': '+ format(round(u0,2)))
    print ('Std Dev of '+ format(values_col)+ ': '+ format(round(sigma,2)))
    
    df_to_return =pd.DataFrame(columns=['Variable','category','count','mean'])
    
    for col in test_columns_list:
        summary_df = df.pivot_table(columns=col,\
                                    values=[values_col], aggfunc={'mean', 'count'})
        summary_df.rename(index=str, columns={values_col: values_col},inplace=True)
        summary_df.columns[0]
        summary_df.columns.name = 'category'
        temp_df = summary_df.transpose().reset_index()
        temp_df['Variable'] = col
        temp_df_2 = pd.concat([temp_df['Variable'],temp_df['category'],
                               temp_df[('age','count')],temp_df[('age','mean')]],
                                  axis = 1, keys = ['Variable','category','count','mean'])
        
        df_to_return = pd.concat([df_to_return,temp_df_2], axis=0)
        df_to_return['ZScore'] = (df_to_return['mean'] - u0)/(sigma/(df_to_return['count']**.5))

        
        
    return df_to_return

In [8]:
titanic_df_updated = titanic_df_raw.copy()
likely_child_func= lambda x: True if x==2 else False
titanic_df_updated['likely_child'] = titanic_df_updated['parch']
titanic_df_updated['likely_child'] = titanic_df_updated['likely_child'].apply(likely_child_func)

In [9]:
titanic_z_df = make_z_test_table(titanic_df_updated, 'age', ['pclass','sex','likely_child','survived','embarked'] )
titanic_z_df.sort_values(['ZScore'])

Median of age: 28.0
Mean of age: 29.88
Std Dev of age: 14.41


Variable     ZScore category  count       mean
2        pclass  -7.865193        3  501.0  24.816367
1  likely_child  -7.451584     True   97.0  18.975979
0           sex  -1.631808   female  388.0  28.687088
1      survived  -1.380458        1  427.0  28.918244
2      embarked  -1.233802        S  782.0  29.245205
1      embarked  -0.613791        Q   50.0  28.630000
1        pclass  -0.419686        2  261.0  29.506705
0      survived   1.146547        0  619.0  30.545363
1           sex   1.253061     male  658.0  30.585228
0  likely_child   2.382327    False  949.0  30.995785
0      embarked   2.475983        C  212.0  32.332170
0        pclass  10.848791        1  284.0  39.159930

### Updating Age Values

With the titanic dataset, The blank age values will be updated using the average values of the groups, if the z_score for that group is above 1.65.  I'll start with the most extreme values of the Z score.  Otherwise, I'll update the value to the average of the population. Also, I'll use a new column and maintain the original values, incase there original values are needed agian.

##### This is was an unproductive rabbit hole.  Perhaps it will be useeful in the future...

Updateing missing values to median, beacause the population skews younger.

In [10]:
titanic_df_updated.age.fillna(titanic_df_updated.age.median(),inplace=True)

## Part 4: Group Statistics

#### Are there any groups that were especially adversely affected in the Titanic wreck? (justify your response numerically)

There are signifcant relationships between the survival rates and group membership on the Titanic.  Of four vairables considered, Sex, Age, Passenger Class and Point of Embarkation.  Point of Embarkation is a strongly related to the passenger class, and can be disimissed.  

Overall, second and third class passengers(42% and 25% survival, respectively) faired much more poorly than first class (62% survival). 

Sex is a very strong predictor for survival rate from the titanic disaster.  Females had a 73% chance of surviving, but males had a 19% chance of surviving.

The survival rate for children (under 15 years old) was 57%; for all other age groups the chance of survival was 36%.

The single worst performing group was adult males from third class.  They had a 13.8% chance of survival vs a 38% chance of survival for the overal population of the Titanic.




In [11]:
titanic_df_updated['Age_bins']= pd.cut(titanic_df_updated['age'],bins = [0,15,30,45,80])

In [12]:
titanic_df_updated[['Age_bins']].head()

Age_bins
0  (15, 30]
1   (0, 15]
2   (0, 15]
3  (15, 30]
4  (15, 30]

In [13]:
def chi_reporting(df, Target_Value_col, test_columns_list):
    import numpy as np
    import pandas as pd
    import scipy.stats as sp
        
    list_to_return = [('Column','Chi_squared','p Value', "ddof")]
    
    for col in test_columns_list:
        c_table = pd.crosstab(df[Target_Value_col],df[col])
        chi_sq, p , ddof,_ = sp.chi2_contingency(c_table)
        list_to_return.append((col, chi_sq,p,ddof))
        

        
    return list_to_return

In [14]:
chi_reporting(titanic_df_updated,'survived',['sex','pclass','embarked','Age_bins'])

[('Column', 'Chi_squared', 'p Value', 'ddof'),
 ('sex', 363.61790843882488, 4.5899249369529454e-81, 1),
 ('pclass', 127.85915643930328, 1.7208259588256052e-28, 2),
 ('embarked', 44.241743071452497, 2.4718809874825629e-10, 2),
 ('Age_bins', 25.982267066503969, 9.6192914303518465e-06, 3)]

The four considered columns all show a significant relationshipwith the survival rate.


In [15]:
c_table = pd.crosstab(titanic_df_updated['survived'], [titanic_df_updated['pclass'],
                      titanic_df_updated['Age_bins'], titanic_df_updated['sex']],margins=True).T

In [17]:
c_table

survived                  0    1   All
pclass Age_bins sex                   
1      (0, 15]  female    1    2     3
                male      0    5     5
       (15, 30] female    1   56    57
                male     42   21    63
       (30, 45] female    1   44    45
                male     28   20    48
       (45, 80] female    2   37    39
                male     48   15    63
2      (0, 15]  female    0   16    16
                male      1   11    12
       (15, 30] female    8   46    54
                male     81    9    90
       (30, 45] female    2   25    27
                male     42    4    46
       (45, 80] female    2    7     9
                male     22    1    23
3      (0, 15]  female   18   19    37
                male     29   13    42
       (15, 30] female   72   74   146
                male    298   49   347
       (30, 45] female   18   11    29
                male     74   13    87
       (45, 80] female    2    2     4
                male     17    0    17
All                     809  500  1309

In [18]:
1-((17+74+298.)/(17+87+347)), 500/1309.


(0.1374722838137472, 0.3819709702062643)

In [19]:
pd.crosstab(titanic_df_updated['pclass'], titanic_df_updated['survived'],margins=True)

survived    0    1   All
pclass                  
1         123  200   323
2         158  119   277
3         528  181   709
All       809  500  1309

In [20]:
pd.crosstab(titanic_df_updated['sex'], titanic_df_updated['survived'],margins=True)

survived    0    1   All
sex                     
female    127  339   466
male      682  161   843
All       809  500  1309

In [21]:
pd.crosstab(titanic_df_updated['Age_bins'], titanic_df_updated['survived'],margins=True)

survived    0    1   All
Age_bins                
(0, 15]    49   66   115
(15, 30]  502  255   757
(30, 45]  165  117   282
(45, 80]   93   62   155
All       809  500  1309

#### Are there any groups that outperformed the survival of the latter group? (justify your response numerically)


First and second class females and male children and were the groups most likely to survive with a 93% chance of surviving

## Part 5:  Comparative Statistics:  Lusitania

In [132]:
lusitania_df_raw['survived'] = lusitania_df_raw['Fate']

#Convert Survived to 0 and 1 for easier calculation.

lusitania_df_raw['survived']=lusitania_df_raw['survived'].str.replace('Saved.+$|Saved','1')
lusitania_df_raw['survived']=lusitania_df_raw['survived'].str.replace('(Lost*)$|Not.+$', '0')
lusitania_df_raw['survived']=lusitania_df_raw['survived'].astype('float')


# Convert ages, dropping non-number values
lusitania_df_raw['Age'].unique()
lusitania_df_raw['age_mod'] = lusitania_df_raw['Age']
lusitania_df_raw['age_mod'] = lusitania_df_raw['age_mod'].str.replace('.+months','1')
lusitania_df_raw['age_mod'] = lusitania_df_raw['age_mod'].str.replace('Infant','1')
lusitania_df_raw['age_mod'] = lusitania_df_raw['age_mod'].str.replace('^(\d\d).+\?*','\\1')
lusitania_df_raw['age_mod'] = lusitania_df_raw['age_mod'].str.replace('^(\d) .+','1')
lusitania_df_raw['age_mod'] = lusitania_df_raw['age_mod'].str.replace('\?','28')
lusitania_df_raw['age_mod'] = lusitania_df_raw['age_mod'].str.replace('(\d)_','\\1')

lusitania_df_raw['age_mod'].unique()
lusitania_df_raw['age_mod'] = lusitania_df_raw['age_mod'].astype('float')
lusitania_df_raw['age_mod'].fillna(32,inplace=True)
lusitania_df_raw['age_bins'] = pd.cut(lusitania_df_raw['age_mod'],bins = [0,15,30,45,80])

#### Are there any groups that were especially adversely affected in the Titanic wreck? (justify your response numerically)

Using the Chi^2 test, sex is not a significant factor. The survival for both sexes is about 38%.

Comparing Sex, age, and passenger class, there is no signicficant difference for survival rates for any group.

In [149]:
#chi_reporting(lusitania_df_raw,'survived',['age_bins'])

sp.chi2_contingency(pd.crosstab(lusitania_df_raw['survived'],lusitania_df_raw['Sex']))
sp.chi2_contingency(pd.crosstab(lusitania_df_raw['survived'],lusitania_df_raw['age_bins']))
pd.crosstab(lusitania_df_raw['survived'],lusitania_df_raw['Sex'],margins=True)
pd.crosstab(lusitania_df_raw['survived'],lusitania_df_raw['Department/Class'],margins=True)


Department/Class  Band  Deck  Engineering  Saloon  Second  Stowaway  Third  \
survived                                                                     
0.0                  2    32          201     178     372         3    236   
1.0                  3    37          112     113     229         0    134   
All                  5    69          313     291     601         3    370   

Department/Class  Third (Distressed British Seaman)  Victualling   All  
survived                                                                
0.0                                               3          167  1194  
1.0                                               0          139   767  
All                                               3          306  1961

#### Are there any groups that outperformed the survival of the latter group? (justify your response numerically)



No.  The average survival rate for all considered groups is about 38%

#### What does the group-wise survival rate imply about circumstances during these two accidents?

For the Lusitania, one factor significantly changed the evacuation of the ship.  Primarily, since the Lusitania was sunk by a torpedo attack, the Lusitania sank much faster than the Titanic.  The rapid sinking led to rapid launching of the lifeboats with any persons hat could board the boat.  The protocol of women and children first was abandoned for filling the lifeboats as quickly as possible.  Only, 6 of the 48 lifeboasts were launched indicating the rapid, but incomplete, evacuation of the ship.

